In [3]:
import pdfminer
import json

read_pdf = None
# f = open("Homework_1.pdf", "rb")
# read_pdf = pdfminer.
print(pdfminer)

<module 'pdfminer' from 'c:\\Users\\chp20\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\pdfminer\\__init__.py'>


In [33]:
#%pip uninstall pdfminer.six
#%pip install pdfminer.six
import pdfminer.high_level as pdf_hl
output = pdf_hl.extract_pages("Homework_1.pdf")
#print(output)
output = list(output)
print(output)
import pdfminer.layout as pdf_layout


[<LTPage(1) 0.000,0.000,612.000,792.000 rotate=0>, <LTPage(2) 0.000,0.000,612.000,792.000 rotate=0>]


In [89]:
import PIL
from PIL import Image
from PIL import ImageDraw
from matplotlib import font_manager
font_cache = {}
def get_font(fontname1,size=12)->PIL.ImageFont.ImageFont:
    global font_cache
    size = int(size)
    font_key = str(fontname1) + str(size)
    if font_key in font_cache:
        return font_cache[font_key]
    fontname = fontname1
    if fontname.find("CM") != -1:
        fontname = "csmy10"
    fontprop = font_manager.FontProperties(family=fontname)
    if fontprop.get_name().find("Deja") != -1:
        fontprop = font_manager.FontProperties(family="Times New Roman")
    fontpath = font_manager.findfont(fontprop)
    font_cache[font_key] = PIL.ImageFont.truetype(fontpath, size)
    return font_cache[font_key]
def mirror_bbox_vertical(bbox, height):
    x0, y0, x1, y1 = bbox
    return (x0, height - y1, x1, height - y0)
def mix_col(c1, c2, alpha):
    c1_a = c1[3] / 255
    c2_a = c2[3] / 255
    tot_a = min(c1_a + c2_a, 1)
    c1_t = tuple([c1[i] * c1[3] / 255 for i in range(3)])
    c2_t = tuple([c2[i] * c2[3] / 255 for i in range(3)])
    c_t = tuple([(c1_t[i] * alpha + c2_t[i] * (1 - alpha)) for i in range(3)] + [tot_a * 255])
    return tuple([int(c_t[i]) for i in range(4)])
class PagePixGrid:
    def __init__(self, page: pdf_layout.LTPage):
        self.page: pdf_layout.LTPage = page
        self.unpacked:bool = False
        self.bbox:tuple = page.bbox
        self.width:int = page.width
        self.height:int = page.height
        self.grid = [[(255,255,255,255) for _ in range(self.width)] for _ in range(self.height)]
        self.bboxes:list[tuple] = []
        self.bbox_contents = []
        self.content_bbox:None|tuple = None #The smallest box that contains all the content
        self.color_enums:dict[str, tuple] = {
            "red" : (255,0,0,255),
            "green" : (0,255,0,255),
            "blue" : (0,0,255,255),
            "white" : (255,255,255,255),
            "black" : (0,0,0,255),
            "yellow" : (255,255,0,255),
            "cyan" : (0,255,255,255),
            "magenta" : (255,0,255,255),
            "transparent_red" : (255,0,0,100),
            "transparent_green" : (0,255,0,100),
            "transparent_blue" : (0,0,255,100),
            "transparent_white" : (255,255,255,100),
            "transparent_black" : (0,0,0,100),
            "transparent_yellow" : (255,255,0,100),
            "transparent_cyan" : (0,255,255,100),
            "transparent_magenta" : (255,0,255,100)
        }
        self.textbox_types = [pdf_layout.LTTextBox, pdf_layout.LTTextBoxHorizontal, pdf_layout.LTTextBoxVertical]
        self.text_line_types = [pdf_layout.LTTextLine, pdf_layout.LTTextLineHorizontal, pdf_layout.LTTextLineVertical]
        self.text_types = [pdf_layout.LTChar]

    def content_bbox_extend(self, x:int, y:int):
        if self.content_bbox is None:
            self.content_bbox = (x, y, x, y)
        else:
            x0, y0, x1, y1 = self.content_bbox
            self.content_bbox = (min(x0, x), min(y0, y), max(x1, x), max(y1, y))
    
    def fill(self, x:int, y:int, color:tuple|None = None):
        if color is None:
            color = (255,0,0,255)
        color:tuple
        if color[3] == 0:
            return
        elif color[3] < 255:
            self.grid[y][x] = mix_col(self.grid[y][x], color, color[3]/255)
        else:
            self.grid[y][x] = color
        self.content_bbox_extend(x, y)
        


    def fill_line(self, x0:int, y0:int, x1:int, y2:int, color:tuple|None = None):
        start:tuple = (x0, y0)
        end:tuple = (x1, y2)
        distVec:tuple = (end[0] - start[0], end[1] - start[1])
        dist:float = max((distVec[0]**2 + distVec[1]**2)**0.5,1)
        distVec:tuple[float,float] = (distVec[0]/dist, distVec[1]/dist)
        for i in range(int(dist+1)):
            self.fill(int(start[0] + i*distVec[0]), int(start[1] + i*distVec[1]), color)


    def round_bbox(self, bbox:tuple)->tuple:
        #Want to round the bbox to the nearest pixels
        #print(bbox)
        x0, y0, x1, y1 = bbox
        x0 = int(x0)
        y0 = int(y0)
        x1 = int(x1)
        y1 = int(y1)
        return (x0, y0, x1, y1)
    
    def fill_bbox(self, bbox:tuple, color:tuple|None = None, fillColor:tuple|None = None):
        if color is None:
            color = (255,0,0,255)
        color:tuple
        x0, y0, x1, y1 = self.round_bbox(bbox)
        top = min(y0, y1)
        bottom = max(y0, y1)
        height = bottom - top
        left = min(x0, x1)
        right = max(x0, x1)
        width = right - left
        self.fill_line(left, top, right, top, color)
        self.fill_line(left, bottom, right, bottom, color)
        self.fill_line(left, top, left, bottom, color)
        self.fill_line(right, top, right, bottom, color)
        if height > 1 and width > 1:
            if fillColor is None:
                fillColor = (color[0], color[1], color[2], 100)
            fillColor:tuple
            for x in range(left+1, right):
                for y in range(top+1, bottom):
                    self.fill(x, y, fillColor)

    def mirror_bbox_vertical(self, bbox:tuple)->tuple:
        return mirror_bbox_vertical(bbox, self.height)
    
    def get_fontdata(self, page_content:pdf_layout.LTComponent)->dict:
        #We are passed some component, we want to recursively check children for font data
        #print(f"Type: {type(page_content)}")
        fontdata = {}
        if type(page_content) in self.textbox_types:
            for child in page_content:
                return self.get_fontdata(child)
        elif type(page_content) in self.text_line_types:
            for child in page_content:
                return self.get_fontdata(child)
        elif type(page_content) in self.text_types:
            if hasattr(page_content, "fontname"):
                fontdata["fontname"] = page_content.fontname
                #print(f"Fontname: {page_content.fontname}")
            if hasattr(page_content, "fontsize"):
                fontdata["fontsize"] = page_content.fontsize
                #print(f"Fontsize: {page_content.fontsize}")
            if hasattr(page_content, "size"):
                fontdata["size"] = page_content.size
                #print(f"Size: {page_content.size}")
        return fontdata

    def add_bbox(self, bbox:tuple, color:tuple|None = None, fillColor:tuple|None = None):
        self.bboxes.append(bbox)
        self.fill_bbox(bbox, color, fillColor)

    def add_bbox_content(self, text:str|None, color:tuple|None = None, fontdata:dict|None = None):
        #bbox already added with add_bbox
        self.bbox_contents.append({"text":text, "color":color, "fontdata":fontdata})

    def push_content(self, page_content:pdf_layout.LTComponent, outlineColor:tuple|None = None, fillColor:tuple|None = None, textColor:tuple|None = None):
        bbox = page_content.bbox
        bbox = self.mirror_bbox_vertical(bbox)
        self.add_bbox(bbox, outlineColor, fillColor)
        fontdata = None
        if type(page_content) in self.textbox_types:
            text = page_content.get_text()
            #print(f"BBox {len(self.bboxes)} has text")
            #get font data
            fontdata = self.get_fontdata(page_content)
            #print(f"Fontdata: {fontdata}")
        else:
            #print(f"Type: {type(page_content)}")
            text = None
        self.add_bbox_content(text, textColor, fontdata)

    def draw(self, bbox_cmp=False, content_bbox=False, draw_text=False)->PIL.Image.Image:
        img = Image.new('RGBA', (self.width, self.height), (255,255,255,255))
        draw = ImageDraw.Draw(img)
        for x in range(self.width):
            for y in range(self.height):
                draw.point((x,y), fill = self.grid[y][x])
        if bbox_cmp:
            for bbox in self.bboxes:
                draw.rectangle(bbox, outline = (0,0,0,255))
        if content_bbox:
            draw.rectangle(self.content_bbox, outline = (0,255,0,255))
        if draw_text:
            for i in range(len(self.bboxes)):
                content = self.bbox_contents[i]
                text = content["text"]
                color = content["color"]
                fontdata = content["fontdata"]
                fontsize = 12
                if fontdata is not None and ("fontsize" in fontdata or "size" in fontdata):
                    if "fontsize" in fontdata:
                        fontsize = fontdata["fontsize"]
                    else:
                        fontsize = fontdata["size"]
                if fontdata is not None and "fontname" in fontdata:
                    font = get_font(fontdata["fontname"], fontsize)
                else:
                    font = get_font("", fontsize)
                if text is not None:
                    draw.text(self.bboxes[i][0:2], text, fill = color, font = font)
        return img
    
    def save(self, filename, *args, **kwargs):
        self.draw(*args, **kwargs).save(filename)

    def unpack_page(self):
        if self.unpacked:
            return
        for box in self.page:
            outlineCol = self.color_enums["red"]
            fillCol = self.color_enums["transparent_red"]
            textCol = self.color_enums["black"]
            self.push_content(box, outlineCol, fillCol, textCol)
        self.unpacked = True

In [81]:
from matplotlib import font_manager
def trimSubsetFont(fontname:str)->str:
    if fontname.find("+") != -1:
        return fontname[fontname.find("+")+1:]
def getFontList(page:pdf_layout.LTPage):
    fontList = []
    for box in page:
        if type(box) in [pdf_layout.LTTextBox, pdf_layout.LTTextBoxHorizontal, pdf_layout.LTTextBoxVertical]:
            for line in box:
                for char in line:
                    if hasattr(char, "fontname"):
                        fontname = trimSubsetFont(char.fontname)
                        if fontname not in fontList:
                            fontList.append(fontname)
    return fontList
# def checkFontInPIL(fontname:str):
#     try:
#         if fontname.find("CM") != -1:
#             fontproperty = font_manager.FontProperties(math_fontfamily="cm", family=fontname)
#         else:
#             fontproperty = font_manager.FontProperties(family=fontname)
#         font = font_manager.findfont(fontproperty)
#         f_font=PIL.ImageFont.truetype(font)
#         print(f_font)
#         print(f_font.getname())
#         return True
#     except:
#         return False
for page in output:
    print(f"Page: {page}")
    flist = getFontList(page)
    for fontname in flist:
        print(f"Font: {fontname}")
        # print(f"Font in PIL: {checkFontInPIL(fontname)}")
# print("matplotlib fonts:")
# allfonts = font_manager.get_font_names()
# fontswithlib = [x for x in allfonts if x.find("Comic Sans MS") != -1]
# print(fontswithlib)
# comic_sans = font_manager.FontProperties(family="Comic Sans MS")
# comic_sans = font_manager.findfont(comic_sans)
# print(comic_sans)
# print(PIL.ImageFont.truetype(comic_sans))

Page: <LTPage(1) 0.000,0.000,612.000,792.000 rotate=0>
Font: LinLibertineTB
('DejaVu Sans', 'Book')
Font in PIL: True
Font: LinLibertineT
('DejaVu Sans', 'Book')
Font in PIL: True
Font: CMMIB10
('DejaVu Sans', 'Book')
Font in PIL: True
Font: CMR12
('DejaVu Sans', 'Book')
Font in PIL: True
Font: CMSY10
('cmsy10', 'Regular')
Font in PIL: True
Font: CMBX12
('DejaVu Sans', 'Book')
Font in PIL: True
Font: CMEX10
('cmex10', 'Regular')
Font in PIL: True
Font: CMMI12
('DejaVu Sans', 'Book')
Font in PIL: True
Font: LinLibertineTI
('DejaVu Sans', 'Book')
Font in PIL: True
Page: <LTPage(2) 0.000,0.000,612.000,792.000 rotate=0>
Font: LinLibertineT
('DejaVu Sans', 'Book')
Font in PIL: True
Font: CMR12
('DejaVu Sans', 'Book')
Font in PIL: True
Font: CMMI12
('DejaVu Sans', 'Book')
Font in PIL: True
Font: CMR8
('DejaVu Sans', 'Book')
Font in PIL: True
Font: CMSY10
('cmsy10', 'Regular')
Font in PIL: True
Font: CMEX10
('cmex10', 'Regular')
Font in PIL: True
Font: CMMIB10
('DejaVu Sans', 'Book')
Font in 

In [90]:
page_handlers = []
for page in output:
    page_handlers.append(PagePixGrid(page))
for ind, page in enumerate(page_handlers):
    page:PagePixGrid
    page.unpack_page()
    page.save(f"page_{ind}.png", bbox_cmp=False, content_bbox=True, draw_text=True)

    

In [6]:
# import PIL
# from PIL import Image
# from PIL import ImageDraw
# #images = []

In [9]:
# #output = list(output)
# images = []
# for page in output:
#     print(page)
#     p_bbox = page.bbox
#     width = p_bbox[2] - p_bbox[0]
#     height = p_bbox[3] - p_bbox[1]
#     p_image = PIL.Image.new('RGB', (width, height), 'white')
#     p_draw = ImageDraw.Draw(p_image)
#     print("-----")
#     for line in page:
#         print(line)
#         bbox = line.bbox
#         print(bbox)
#         width_line = bbox[2] - bbox[0]
#         height_line = bbox[3] - bbox[1]
#         x0 = bbox[0] - p_bbox[0]
#         x1 = bbox[2] - p_bbox[0]
#         #y coordinates are flipped
#         y0 = height - (bbox[3] - p_bbox[1])
#         y1 = height - (bbox[1] - p_bbox[1])
#         p_draw.rectangle([x0, y0, x1, y1], outline='red')
#         #p_draw.text((bbox[0] - p_bbox[0], bbox[1] - p_bbox[1]), line.get_text(), fill='black')
#     images.append(p_image)
#     #p_image.show()
#     #break

# for ind, img in enumerate(images):
#     img.save(f"page_{ind}.png", "PNG")

<LTPage(1) 0.000,0.000,612.000,792.000 rotate=0>
-----
<LTTextBoxHorizontal(0) 77.978,678.246,181.068,719.094 'MATH 420\nHomework 1\nSeptember 20, 2023\n'>
(77.978, 678.246304, 181.0676896, 719.093504)
<LTTextBoxHorizontal(1) 394.289,707.138,428.911,719.094 'Name:\n'>
(394.2886816, 707.1383040000001, 428.91094080000005, 719.093504)
<LTTextBoxHorizontal(2) 215.679,636.222,396.322,648.178 'Submit your work on separate paper.\n'>
(215.679, 636.2224384, 396.322072, 648.1776384000001)
<LTTextBoxHorizontal(3) 87.223,608.825,452.965,621.247 '1. Find a certificate of infeasibility for the system Ax = b, x ≥ 0 given by\n'>
(87.22300000000001, 608.8254383999999, 452.9648864, 621.2468911999999)
<LTTextBoxHorizontal(4) 240.969,588.013,248.940,599.968 '\uf8ee\n'>
(240.96900000000002, 588.0128799999999, 248.93953184000003, 599.9680799999999)
<LTTextBoxHorizontal(5) 240.969,566.494,248.940,578.449 '\uf8f0\n'>
(240.96900000000002, 566.4938799999999, 248.93953184000003, 578.4490799999999)
<LTTextBoxHor